<a href="https://colab.research.google.com/github/Atomix77/IMLO-IP/blob/main/IMLO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load Data
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from torch.optim import lr_scheduler

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trainingTransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomRotation(180),
    transforms.RandomAutocontrast(),
    transforms.RandomAdjustSharpness(1.5, 0.5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    ])

validationTransform = transforms.Compose([ 
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

testTransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

trainingData = datasets.Flowers102(
    root = "./datasets",
    split = "train",
    transform = trainingTransform,
    download = True)

validationData = datasets.Flowers102(
    root = "./datasets",
    split = "val",
    transform = validationTransform,
    download = True)

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    download = True)

100%|███████████████████████| 344862509/344862509 [00:11<00:00, 29678530.90it/s]


Extracting datasets/flowers-102/102flowers.tgz to datasets/flowers-102


100%|█████████████████████████████████████| 502/502 [00:00<00:00, 382061.44it/s]


100%|████████████████████████████████| 14989/14989 [00:00<00:00, 4561632.76it/s]


In [2]:
# Dataloaders
trainingDataloader = DataLoader(trainingData, batch_size = 8, shuffle = True, num_workers = 6)
validationDataloader = DataLoader(validationData, batch_size = 32, shuffle = False, num_workers = 6)
testDataloader = DataLoader(testData, batch_size = 32, shuffle = False, num_workers = 6)

In [3]:
# Get device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# Create Aritecture
class NeuralNetwork(nn.Module):
    def __init__(self, classAmount):
        super(NeuralNetwork, self).__init__()
        self.convStack =  nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(32, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),          

            nn.Conv2d(64, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Dropout(0.5)
        )

        dummyInput = torch.zeros(1, 3, 256, 256)
        dummyOutput = self.convStack(dummyInput)
        self.convOutputSize = dummyOutput.view(1, -1).size(1)
        print(self.convOutputSize)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.convOutputSize, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),

            nn.Linear(2048, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.ReLU(),
            
            nn.Linear(512, classAmount)
        )
    

    def forward(self, x):
        x = self.convStack(x)
        x = self.classifier(x)
        return x

model = NeuralNetwork(102).to(device)

512


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [5]:
# Train Model
losses = []
accuracy = []
def trainModel(dataloader, model, lossFunction, optimizer):
    model.train()
    currentLoss = 0.0
    correct = 0
    total = 0
    epochLoss = 0
    
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        total += y.size(0)

        optimizer.zero_grad()
        pred = model(X)
        loss = lossFunction(pred, y)
        loss.backward()
        optimizer.step()

        currentLoss += loss.item()
        epochLoss += lossFunction(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    epochLoss = epochLoss/total
    correct = correct/total

    losses.append(epochLoss)
    accuracy.append(correct * 100)
    print(f'Training: Accuracy {correct * 100:>0.1f}%, Loss: {currentLoss / len(dataloader):.5f}, Epoch Loss: {epochLoss:.5f}')

In [6]:
# Validate Model
def validateModel(dataloader, model, lossFunction):
    model.eval()
    total = 0
    correct = 0
    epochLoss = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total += y.size(0)
            epochLoss += lossFunction(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    epochLoss = epochLoss/total
    correct = (correct/total) * 100

    print(f"Validation: Accuracy: {(correct):>0.1f}%, Avg loss: {epochLoss:>8f} \n")
    return epochLoss, correct

In [7]:
#Test Model
def testModel(dataloader, model, lossFunction):
    model.eval()
    total = 0
    correct = 0
    testLoss = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total += y.size(0)
            testLoss += lossFunction(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    testLoss = testLoss/total
    correct = (correct/total) * 100
    print(f"Testing: Accuracy: {(correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")
    return testLoss, correct

In [8]:
# Model Evaluation
learningRate = 0.0001
weightDecay = 0.0001
epochs = 1000

bestValAccuracy = 0.0
bestValLoss = float('inf')
bestEpoch = 0

valLosses = []
valAccuracies = []

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = lr_scheduler.StepLR(optimizer, step_size=450, gamma=0.99)

for epoch in range(epochs):
    print(f'Epoch {epoch+1}:')
    trainModel(trainingDataloader, model, lossFunction, optimizer)
    valLoss, valAccuracy = validateModel(validationDataloader, model, lossFunction)
    valLosses.append(valLoss)
    valAccuracies.append(valAccuracy)
    scheduler.step()

    if (valLoss < bestValLoss):
        bestValLoss = valLoss
        bestValAccuracy = valAccuracy
        bestEpoch = epoch+1

testModel(testDataloader, model, lossFunction)
print(f'Best Accuracy: {bestValAccuracy}. Best Loss: {bestValLoss} Best Epoch: {bestEpoch}')
print("Finished")

# Save model
torch.save(model.state_dict(),'bestModel.pt')

Epoch 1:
Training: Accuracy 0.7%, Loss: 4.82547, Epoch Loss: 0.60555
Validation: Accuracy: 0.6%, Avg loss: 0.144152 

Epoch 2:
Training: Accuracy 1.0%, Loss: 4.72876, Epoch Loss: 0.59341
Validation: Accuracy: 2.4%, Avg loss: 0.141683 

Epoch 3:
Training: Accuracy 0.6%, Loss: 4.68575, Epoch Loss: 0.58802
Validation: Accuracy: 3.2%, Avg loss: 0.138922 

Epoch 4:
Training: Accuracy 2.6%, Loss: 4.58092, Epoch Loss: 0.57486
Validation: Accuracy: 2.9%, Avg loss: 0.136312 

Epoch 5:
Training: Accuracy 2.8%, Loss: 4.50350, Epoch Loss: 0.56515
Validation: Accuracy: 4.0%, Avg loss: 0.135507 

Epoch 6:
Training: Accuracy 2.1%, Loss: 4.44768, Epoch Loss: 0.55814
Validation: Accuracy: 3.8%, Avg loss: 0.130524 

Epoch 7:
Training: Accuracy 2.5%, Loss: 4.36360, Epoch Loss: 0.54759
Validation: Accuracy: 4.9%, Avg loss: 0.129982 

Epoch 8:
Training: Accuracy 3.1%, Loss: 4.32853, Epoch Loss: 0.54319
Validation: Accuracy: 5.1%, Avg loss: 0.125417 

Epoch 9:
Training: Accuracy 3.6%, Loss: 4.27639, Epoch L

KeyboardInterrupt: 

Testing: Accuracy: 75.2%, Avg loss: 0.032671 - 1000 epochs (434 mins - 7 hrs 12 mins)

Testing: Accuracy: 73.9%, Avg loss: 0.034663 - 1000 epochs (416 mins - 6 hrs 55 mins)

Testing: Accuracy: 72.8%, Avg loss: 0.036552 - 1000 epochs (426 mins - 7 hrs 6 mins)

Testing: Accuracy: 66.2%, Avg loss: 0.041491 - 1000 epochs (619 mins - 10 hrs 20 mins)

Testing: Accuracy 62.7% Avg loss: 0.053697 - 500 epochs (3hrs 30 mins)

Testing: Accuracy: 66.4%, Avg loss: 0.044697 - 500 epochs (3hrs 30 mins)

Testing: Accuracy: 59.3%, Avg loss: 0.060451 - 500 epochs (3 hrs 25 mins)- potentially done with 350 epochs. 

Testing: Accuracy: 48.1%, Avg loss: 0.069918  - 200 epochs

Testing: Accuracy: 48.5%, Avg loss: 0.063186 - 200 epochs

In [4]:
#Load and Test the trained model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

testTransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    download = True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self, classAmount):
        super(NeuralNetwork, self).__init__()
        self.convStack =  nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(32, 32, 3, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),          

            nn.Conv2d(64, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 128, 3, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 256, 3, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, 3, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Dropout(0.5)
        )

        dummyInput = torch.zeros(1, 3, 256, 256)
        dummyOutput = self.convStack(dummyInput)
        self.convOutputSize = dummyOutput.view(1, -1).size(1)
        print(self.convOutputSize)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.convOutputSize, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),

            nn.Linear(2048, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.ReLU(),
            
            nn.Linear(512, classAmount)
        )
    

    def forward(self, x):
        x = self.convStack(x)
        x = self.classifier(x)
        return x

newModel = NeuralNetwork(102).to(device)

newModel.load_state_dict(torch.load('Model.pt', map_location=torch.device('cpu')))
newModel.eval()

def testTrainedModel(dataloader, model, lossFunction):
    model.eval()
    total = 0
    correct = 0
    testLoss = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total += y.size(0)
            testLoss += lossFunction(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    testLoss = testLoss/total
    correct = (correct/total)  * 100
    print(f"Testing: Accuracy: {(correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")

testDataloader = DataLoader(testData, batch_size = 32, shuffle = False, num_workers = 6)
lossFunction = nn.CrossEntropyLoss()

testTrainedModel(testDataloader, newModel, lossFunction)

Using cpu device
512
Testing: Accuracy: 73.9%, Avg loss: 0.034648 

